In [ ]:
import io
import numpy as np
import onnx
import os

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from torch import nn
import torch.onnx

In [ ]:
import onnxruntime

In [ ]:
from torchvision import datasets, transforms
import torch.nn.functional as F

In [ ]:
from torch.utils.data import dataset

In [ ]:
class ConvMNIST(nn.Module):
    def __init__(self):
        super(ConvMNIST, self).__init__()  # Indicates that the ConvMNIST class inherits properties from nn.Module.
        
        self.conv1 = nn.Conv2d(1,32,3,1,1)      # 28*28*3
        self.conv2 = nn.Conv2d(32,64,3,1,1) 
        self.pool = nn.MaxPool2d(kernel_size=2) # 14*14*32
        self.fc2 = nn.Linear(64*14*14,1000)
        self.fc3 = nn.Linear(1000,256)
        self.fc4 = nn.Linear(256,10)
        self.dropout = nn.Dropout(p=0.4)
        
    def forward(self,x):
        
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1,64*14*14)
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))
        x = F.log_softmax(self.fc4(x),dim=1)
        
        return x

In [ ]:
onnx_model = ConvMNIST()

In [ ]:
# List all the saved models
os.listdir('mnist_model_new/')

In [ ]:
# Path to the saved models
path = ''

In [ ]:
# Load and instantiate the model
onnx_model = torch.load(path,map_location=torch.device('cpu'))

In [ ]:
# Setting the model to eval to ensure normal operation of dropouts.
onnx_model.eval()               


We first trace our model to export it to the onnx format. This is done by passing a dummy variable (of proper size) as input to the model. The onnx model also stores the learned parameters and the various operations that contribute to the output. The input size will be fixed in the exported ONNX graph for all the input dimensions. If the model has variable parameters like batch_size, we can specify that as dynamic axes.

In [ ]:
# Change the model name to be saved in onnx format
onnx_model_name = 'MnistConv.onnx'

In [ ]:
batch_size = 1 # set as a random number initially (it is set as dynamic axes later)
# Input to the model
x = torch.randn(batch_size, 1, 28, 28, requires_grad=True) 
torch_out = onnx_model(x)    # storing the model output to compare with the onnx model output

# Export the model
torch.onnx.export(onnx_model,                # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  onnx_model_name,      # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable lenght axes 
                                'output' : {0 : 'batch_size'}})

In [ ]:
model_onnx = onnx.load(onnx_model_name)
onnx.checker.check_model(model_onnx)

In [ ]:
ort_session = onnxruntime.InferenceSession(onnx_model_name)

def to_numpy(tensor):
    return tensor.detach().numpy() if tensor.requires_grad else tensor.numpy()

# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
ort_outs = ort_session.run(None, ort_inputs)

# compare ONNX Runtime and PyTorch results
# ort_outs gives a list of outputs. We are selecting the first one, since we have one output.
np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)

print("Exported model has been tested with ONNXRuntime, and the result is valid")

## Testing 

In [ ]:
os.listdir('handwritten-digits-images-dataset/handwritten-digits-images-dataset/dataset/')

In [ ]:
# path to the test data
root = 'handwritten-digits-images-dataset/handwritten-digits-images-dataset/dataset/'

In [ ]:
transform = transforms.Compose([transforms.Grayscale(),
                                transforms.ToTensor()
                                #transforms.Normalize((0.5,), (0.5,)),
                               ])


testset = datasets.ImageFolder(root+'test', transform=transform)

testloader = torch.utils.data.DataLoader(testset,batch_size=64, shuffle = True)

In [ ]:
# Passing a single image
# Since we have set dynamic axes for batch size, it tells the onnxruntime that batch_size can vary.
# We can pass any number of images.
data = iter(testloader)
img, label = data.next()

In [ ]:
#img = img.numpy()

In [ ]:
img.shape

In [ ]:
# For gray scale image inputs
img = img[5].unsqueeze(dim=0)

In [ ]:
img.shape

In [ ]:
type(img)

In [ ]:
# Gray scale image plot
plt.imshow(img.squeeze())

In [ ]:
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(img)}     # The input must be a numpy array 
ort_outs = ort_session.run(None, ort_inputs)
predicted = ort_outs[0]

In [ ]:
predicted

In [ ]:
a =list(np.exp(predicted).squeeze())
index = a.index(max(a))

In [ ]:
print(f"Predicted class {index}")
print(label[5])